In [ ]:
! pip install yfinance crewai crewai-tools

In [ ]:
! pip install pandas matplotlib yfinance

In [3]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool
import os
from dotenv import load_dotenv

load_dotenv()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


True

In [4]:
from pydantic import BaseModel, Field

class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")

In [5]:
llm =LLM(
    model= "gpt-4.1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# 1) Query parser agent
query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)


# 2) Code writer agent
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data. (first ensure to install the required libraries using pip install yfinance pandas matplotlib)",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)


# 3) Code interpreter agent (uses code interpreter tool from crewai)
code_interpreter_tool = CodeInterpreterTool()

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review and execute the generated Python code by code writer agent to visualize stock data.",
    backstory="You are a code execution expert. You are skilled at executing Python code.",
    # tools=[code_interpreter_tool],
    allow_code_execution=True,   # This automatically adds the CodeInterpreterTool
    code_execution_mode="unsafe",
    llm=llm,
    verbose=True,
    
)

code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)

In [6]:
### --- CREW SETUP --- ###

crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

🖇 AgentOps: Could not initialize AgentOps client - API Key is missing.
	    Find your API key at https://app.agentops.ai/settings/projects


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Task: Analyze the user query and extract stock details.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Exception in thread Thread-6 (check_agentops_update):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/pulkitrajput/Library/Python/3.11/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/agentops/helpers.py", line 132, in check_agentops_update
    conn.request("GET", "/pypi/agentops/json")
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1303, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 13

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "symbol": "TSLA",                                                                                            │
│    "timeframe": "YTD",                                                                                          │
│    "action": "plot gain"                                                                                        │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Thought: Thought: The code is clean, structured, and aims to fetch and plot the YTD gain for TSLA. I should    │
│  run the script to ensure it executes, imports the correct libraries, fetches valid data, and produces the      │
│  expected plot. There are no obvious issues with the dependencies or logic, so I will proceed with code         │
│  execution.                                                                                                     │
│                                                                                                                 │
│  Using Tool: Code Interpreter                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"code\": \"# stock_visualization.py\\n\\n# Ensure required libraries are installed:\\n# pip install yfinan  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  No result variable found.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='```\nTho...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # stock_visualization.py                                                                                       │
│                                                                                                                 │
│  # Ensure required libraries are installed:                                                                     │
│  # pip install yfinance pandas matplotlib                                                                       │
│                                                                                                                 │
│  import yfinance as yf                                                                                          │
│  import pandas as pd                                                                                            │
│  import matplotlib.pyplot as plt                                                                                │
│  from datetime import datetime                                                                                  │
│                                                                                                                 │
│  # --- User Inputs from Analyst ---                                                                             │
│  symbol = "TSLA"                                                                                                │
│  timeframe = "YTD"                                                                                              │
│  action = "plot gain"                                                                                           │
│                                                                                                                 │
│  # --- Helper to determine date range based on timeframe ---                                                    │
│  def get_date_range(timeframe):                                                                                 │
│      today = pd.Timestamp.today().normalize()                                                                   │
│      if timeframe.upper() == "YTD":                                                                             │
│          start = today.replace(month=1, day=1)                                                                  │
│          end = today                                                                                            │
│      else:                                                                                                      │
│          raise ValueError("Only 'YTD' timeframe is implemented in this script.")                                │
│      return start, end                                                                                          │
│                                                                                                                 │
│  # --- Download stock data ---                                                                                  │
│  def fetch_stock_data(symbol, start, end):                                                                      │
│      df = yf.download(symbol, start=start, end=end)                                                             │
│      if df.empty:                                                                                               │
│          raise ValueError(f"No data fetched for {symbol

In [7]:
print(result.raw)

# stock_visualization.py

# Ensure required libraries are installed:
# pip install yfinance pandas matplotlib

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# --- User Inputs from Analyst ---
symbol = "TSLA"
timeframe = "YTD"
action = "plot gain"

# --- Helper to determine date range based on timeframe ---
def get_date_range(timeframe):
    today = pd.Timestamp.today().normalize()
    if timeframe.upper() == "YTD":
        start = today.replace(month=1, day=1)
        end = today
    else:
        raise ValueError("Only 'YTD' timeframe is implemented in this script.")
    return start, end

# --- Download stock data ---
def fetch_stock_data(symbol, start, end):
    df = yf.download(symbol, start=start, end=end)
    if df.empty:
        raise ValueError(f"No data fetched for {symbol} between {start} and {end}.")
    return df

# --- Plotting Gain Chart ---
def plot_stock_gain(df, symbol, timeframe):
    # Gain = (Close - Close[0